# Template generation

The generation of templates is not directly controlled by the end-user. Instead, s-BART provides two interfaces that will automatically sort the observations (in the different sub-Instruments) and create independent stellar- and telluric-templates for each available sub-Instrument.
 Furthermore, the goal of these two interfaces (stellar and telluric) is to create the templates and, afterwards, the DataClass will load their relevant information

In [1]:
##
#   We start by loading some observations
##
from pathlib import Path
import os

resources_path = Path(os.curdir).absolute().parent.parent / "tests/resources"
input_filepath = [i.as_posix() for i in resources_path.iterdir() if "fits" in i.name]
from SBART.data_objects import DataClassManager
from SBART.Instruments import ESPRESSO
manager = DataClassManager()
manager.start()


data = manager.DataClass(
    input_filepath,
    storage_path = "",
    instrument=ESPRESSO,
    instrument_options={},
)

FileNotFoundError: [Errno 2] No such file or directory: '/home/amiguel/phd/tests/resources'

## Handling the telluric features


In [ ]:
from SBART.template_creation.TelluricModel import TelluricModel

# This will be used to control the application of the selected algorithm to the different subInstruments
telluric_model_configs = {"CREATION_MODE": "tapas"}

ModelTell = TelluricModel(
        usage_mode="individual",
        user_configs=telluric_model_configs,
        root_folder_path="",
    )

# The telluric_configs will be used to configure the algorithm that was selected for the construction of the temlate
telluric_template_configs = {"continuum_percentage_drop": 1,
                             "download_path": (resources_path / "tapas.ipac").as_posix()
                             }
ModelTell.Generate_Model(
    dataClass=data,
    telluric_configs=telluric_template_configs,
    force_computation=False,
    store_templates=True,
)

# After creating the telluric model, we load it into the DataClass.
data.remove_telluric_features(ModelTell)

first_frame = data.get_frame_by_ID(0)
wave, flux, _ ,mask = first_frame.get_data_from_spectral_order(147)
import matplotlib.pyplot as plt
plt.scatter(wave[~mask], flux[~mask], color = "black", s=9, label = "Spectra")
plt.scatter(wave[mask], flux[mask], color = "red", s=9, label = "Telluric-rejected features")
plt.legend(loc=4, bbox_to_anchor=(1,1), ncol=2)
plt.show()

## Stellar Model